# deep-muse (ver 0.1) [WIP]

***

# Advanced text-to-music generator

***

## Inspired by https://github.com/lucidrains/deep-daze

## Powered by tegridy-tools TMIDI 2.2 Optimus Processors

***

### Project Los Angeles
### Tegridy Code 2021

***

# Setup environment

In [ ]:
#@title Install dependencies
!git clone https://github.com/asigalov61/tegridy-tools
!pip install tqdm
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio

# packages below are for plotting pianoroll only
# they are not needed for anything else
!pip install pretty_midi
!pip install librosa
!pip install matplotlib

In [ ]:
#@title Load needed modules
print('Loading needed modules. Please wait...')

import sys
import os
import json
import secrets

os.chdir('/content/tegridy-tools/tegridy-tools/')
import TMIDI
os.chdir('/content/')

from pprint import pprint

import tqdm.auto

from midi2audio import FluidSynth
from IPython.display import display, Javascript, HTML, Audio

# only for plotting pianoroll
import pretty_midi
import librosa.display
import matplotlib.pyplot as plt

from google.colab import output, drive

print('Creating Dataset dir...')
if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

os.chdir('/content/')
print('Loading complete. Enjoy! :)')

# Prep statistics dictionary

In [ ]:
#@title Load English Karaoke Statistics
data = TMIDI.Tegridy_Any_Pickle_File_Loader('/content/tegridy-tools/tegridy-data/Basic-English-Karaoke-Statistics')
karp = data[0]
kard = data[1]
karw = data[2]
pitches_words = data[3]
words_music = data[4]

# Generate Music

## NOTE: Karaoke melody only for now

In [ ]:
#@title Generate Music from the lyrics below

#@markdown NOTE: No spaces, commas, etc., please.

#@markdown ProTip: Be as ambiguous and general as possible for best results as the current dictionary is too small for anything specific. 
lyric1 = 'I love you very very much' #@param {type:"string"}
lyric2 = 'I can not live without you' #@param {type:"string"}
lyric3 = 'You always present on my mind' #@param {type:"string"}
lyric4 = 'I often think about you' #@param {type:"string"}

lyric5 = 'I am all out of love I am so lost without you' #@param {type:"string"}
lyric6 = 'I know you were right believing for so long' #@param {type:"string"}
lyric7 = 'I am all out of love what am I without you' #@param {type:"string"}
lyric8 = 'I cant be too late to say that I was so wrong' #@param {type:"string"}

txt = '| ' + lyric1 + ' | ' + lyric2 + ' | ' + lyric3 + ' | ' + lyric4 + ' | ' + lyric5 + ' | ' + lyric6 + ' | ' + lyric7 + ' | ' + lyric8 + ' |'

import copy
txts = txt.split()
song = []
for t in txts:
  try:
    x = copy.deepcopy(words_music[t])
    x.append(t)
    song.append(x)
  except:
    # print('Unknown word', t)
    song.append([0, 1600, '====='])
    continue

pprint(song, compact=True)    

# Convert generated music composition to MIDI file and download/listen to the output :)

In [ ]:
#@title Convert to MIDI
out = []
time = 0
for s in song:
  note = ['note', int(time), s[1], 0, s[0], 90 ]
  out.append(note)
  
  kar = ['lyric', int(time), s[2] ]
  out.append(kar)

  time += s[1]
TMIDI.Tegridy_SONG_to_MIDI_Converter(out, output_file_name='/content/deep-muse-Output-MIDI')

In [ ]:
#@title Plot and listen to the last generated composition
#@markdown NOTE: May be very slow with the long compositions
fname = '/content/deep-muse-Output-MIDI'

fn = os.path.basename(fname + '.mid')
fn1 = fn.split('.')[0]
print('Playing and plotting composition...')

pm = pretty_midi.PrettyMIDI(fname + '.mid')

# Retrieve piano roll of the MIDI file
piano_roll = pm.get_piano_roll()

plt.figure(figsize=(14, 5))
librosa.display.specshow(piano_roll, x_axis='time', y_axis='cqt_note', sr=64000, cmap=plt.cm.hot)
plt.title('Composition: ' + fn1)

print('Synthesizing the last output MIDI. Please stand-by... ')
FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname + '.mid'), str(fname + '.wav'))
Audio(str(fname + '.wav'), rate=16000)

# English Karaoke Statistics Graphs

In [ ]:
#@title Pitches-Words Graph
from matplotlib import pyplot as plt
from matplotlib import style
 
style.use('ggplot')
 
x = pitches_words.values()
y = pitches_words.keys()
 
plt.bar(x, y, align='center')
 
plt.title('Pitches Words Diagram')
plt.ylabel('Pitches')
plt.xlabel('Words')
 
plt.show()

In [ ]:
#@title Pitches-Words Graph 2
colors = list("rgb")


x = pitches_words.keys()
y = pitches_words.values()
plt.scatter(x,y,color=colors.pop())

plt.legend(pitches_words.keys())
plt.savefig('/content/scatterplot.png', dpi=300)
plt.show()

In [ ]:
#@title Pitches-Words Graph 3
labels, values = zip(*pitches_words.items())
plt.bar(labels, values)

In [ ]:
#@title Pitches-Words Graph 4
plt.bar(range(len(pitches_words)), pitches_words.values(), align="center")
plt.xticks(range(len(pitches_words)), list(pitches_words.keys()))

# Congrats! You did it! :)